In [1]:
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency
from scipy.stats import chi2
from scipy.stats import norm

## Question 8

Investigate following mortality patterns

| week | chinese | jewish |
|------|---------|--------|
| -2   | 55      | 141    |
| -1   | 33      | 145    |
| 1    | 70      | 139    |
| 2    | 49      | 161    |

Table shows number of old women who died in the weeks before and after the Chinese Harvest Moon Festival

In [2]:
data = np.array([
    [55, 141],
    [33, 145],
    [70, 139],
    [49, 161]
])

### Test independence between race and week of death using chi2

In [3]:
chi2_contingency(data)

(12.420794151882923,
 0.006072304928267781,
 3,
 array([[ 51.16267339, 144.83732661],
        [ 46.46406053, 131.53593947],
        [ 54.55611602, 154.44388398],
        [ 54.81715006, 155.18284994]]))

p-value of 0.006 shows there is strong evidence that that race and week of death are dependent

### test that number of deaths for chinese people is independent of week

Let pi = probability of death in each week. H0: pi = 0.25 for all i

In [4]:
chinese_deaths = data[:,0]
chinese_deaths

array([55, 33, 70, 49])

In [5]:
chinese_deaths_expected = np.full((4,), 0.25*chinese_deaths.sum())
chinese_deaths_expected

array([51.75, 51.75, 51.75, 51.75])

In [6]:
test_statistic = (
    (chinese_deaths - chinese_deaths_expected) ** 2 / chinese_deaths_expected
).sum()

test_statistic

13.579710144927537

In [7]:
p_value = 1-chi2.cdf(test_statistic, df=3)
p_value

0.0035368204078958243

p-value of 0.003 is strong evidence that probability of death for chinese women depends on week number.

### test that number of deaths for jewish people is independent of week

Let pi = probability of death in each week. H0: pi = 0.25 for all i

In [8]:
jewish_deaths = data[:,1]
jewish_deaths

array([141, 145, 139, 161])

In [9]:
jewish_deaths_expected = np.full((4,), 0.25*jewish_deaths.sum())
jewish_deaths_expected

array([146.5, 146.5, 146.5, 146.5])

In [10]:
test_statistic = (
    (jewish_deaths - jewish_deaths_expected) ** 2 / jewish_deaths_expected
).sum()

test_statistic

2.040955631399317

In [11]:
p_value = 1-chi2.cdf(test_statistic, df=3)
p_value

0.5639498711784423

p-value of 0.56 is evidence that number of deaths for jewish women is independent of week number

## Question 9
Conducting hypothesis tests that a new drugs are better than placebo at reducing nausea

|                        | Number of Patients | Incidence of Nausea |
|------------------------|--------------------|---------------------|
| Placebo                | 80                 | 45                  |
| Chlorpromazine         | 75                 | 26                  |
| Dimenhydrinate         | 85                 | 52                  |
| Pentobarbital (100 mg) | 67                 | 35                  |
| Pentobarbital (150 mg) | 85                 | 37                  |

In [12]:
data = np.array([
    [80, 45],
    [75, 26],
    [85, 52],
    [67, 35],
    [85, 37],
])

# recalculate second column to correspond to successes, rather than failures
data[:,1] = data[:,0] - data[:,1]

print(data)

[[80 35]
 [75 49]
 [85 33]
 [67 32]
 [85 48]]


In [13]:
def test_p2_gr_p1(n1, x1, n2, x2):
    """
    behaviour:
        conduct one-sided wald test
        assume X1 ~ Bin(n1, p1), X2 ~ Bin(n2, p2) and X1 and X2 are independent
        H0: p1=p2, H1: p2>p1
        test statistic is t = p2_hat - p1_hat
        under h0, t  approximately follows N(0, var(p1_hat)+var(p2_hat))
    
    inputs:
        n1 and n2: parameters of distributions X1 and X2
        x1, x2: observed instances of X1 and X2
    
    output:
        results dictionary containing
            p1_hat
            p2_hat
            t
            se
            p-value
    
   
    """
    p1_hat = x1/n1
    p2_hat = x2/n2
    
    t = p2_hat - p1_hat
    
    se = (p1_hat*(1-p1_hat)/n1 + p2_hat*(1-p2_hat)/n2)**0.5
    
    Z = t/se
    
    p_value = 1 - norm.cdf(Z)
    
    results = dict(
        p1_hat = p1_hat,
        p2_hat = p2_hat,
        t = t,
        se = se,
        p_value = p_value
    )
    
    return results

In [14]:
for i in range(1,5):
    print(f'Treatment {i}')
    print(test_p2_gr_p1(data[0,0], data[0,1], data[i,0], data[i,1]))
    print()

Treatment 1
{'p1_hat': 0.4375, 'p2_hat': 0.6533333333333333, 't': 0.21583333333333332, 'se': 0.07807703712905512, 'p_value': 0.002851695777628893}

Treatment 2
{'p1_hat': 0.4375, 'p2_hat': 0.38823529411764707, 't': -0.04926470588235293, 'se': 0.07661847879507237, 'p_value': 0.7398838172885266}

Treatment 3
{'p1_hat': 0.4375, 'p2_hat': 0.47761194029850745, 't': 0.040111940298507454, 'se': 0.08246231979025985, 'p_value': 0.31333204736569453}

Treatment 4
{'p1_hat': 0.4375, 'p2_hat': 0.5647058823529412, 't': 0.12720588235294117, 'se': 0.0772534224000257, 'p_value': 0.04981961670910118}



p-values are 0.0028, 0.739, 0.313, 0.049

#### No corrections for multiple tests
at 5% signifiance level, we would conclude that treatment 1 and treatment 4 have significant results.

#### Bonferonni corrections
cutoff becomes 0.0125 rather than 0.05, so only treatment 1 would be considered significant

#### Benjamni-Hochberg:
ordered p-values: 0.0028, 0.049, 0.313, 0.739

compare p-values to: 0.0125, 0.025, 0.0375, 0.05

get same conclusion as bonferonni: only consider treatment 1 significant.

#### Final conclusions
strong evidence that treatment 1 is significant, some evidence that treatment 4 is significant